# ¿Cómo viajan los turistas en España? 
### Un estudio de tendencias y comportamiento

### Librerias utilizadas

In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import random

### Importacion de DataSet

In [3]:
turismo = pd.read_excel("./src/data/tourism_dataset.xlsx")
tipologia = pd.read_excel("./src/data/tipologia_viaje.xlsx")
motivo = pd.read_excel("./src/data/motivo_viaje.xlsx")
pais = pd.read_excel("./src/data/Gasto_segun_paisxlsx.xlsx")
sector = pd.read_excel("./src/data/Gasto_por_sector.xlsx")

### Verificacion de tipos

In [4]:
# print(turismo.info())
# print(tipologia.info())
# print(motivo.info())
# print(pais.info())
# print(sector.info())

### Convertir Dataset

In [5]:
turismo = pd.DataFrame(turismo)
tipologia = pd.DataFrame(tipologia)
motivo = pd.DataFrame(motivo)
pais = pd.DataFrame(pais)
sector = pd.DataFrame(sector)

# turismo.to_csv('./data/datos_turismo.csv')
# tipologia.to_csv('./data/datos_tipologia.csv')
# motivo.to_csv('./data/datos_motivo.csv')
# pais.to_csv('./data/datos_gasto_pais.csv')
# sector.to_csv('./data/datos_sector.csv')

### Reorganizacion del Dataset:
* **Columnas**: Se restablece el nombre de las columnas, con el nombre de las comunidades autonomas.
* **Indices**: Los indices son los meses y años

In [6]:
turismo = turismo.set_index(turismo.columns[0]).transpose()
tipologia = tipologia.set_index(tipologia.columns[0])
motivo = motivo.set_index(motivo.columns[0])
pais = pais.set_index(pais.columns[0])
sector = sector.set_index(sector.columns[0])

### Despues de la comprobacion del Dataset, hacemos un .Dtypes y el resultado es de "object" los cuales debemos convertir a "float", para poder trabajar con los numeros

In [7]:
# print(turismo.dtypes)
# print(tipologia.dtypes)
# print(motivo.dtypes)
# print(pais.dtypes)
# print(sector.dtypes)

In [ ]:
for col in turismo.columns:
    turismo[col] = pd.to_numeric(turismo[col], errors="coerce") #Esto convierte algunos valores a float y otros a int.
turismo = turismo.astype(float) #Convierte el restante a float para asegurar que todos los datos tengan el mismo tipo.

for tipo in tipologia.columns:
    tipologia[tipo] = pd.to_numeric(tipologia[tipo], errors="coerce")
pd.set_option("display.float_format", "{:.2f}".format)
tipologia = tipologia.astype(float)

for col in motivo.columns:
    motivo[col] = pd.to_numeric(motivo[col], errors="coerce")
pd.set_option("display.float_format", "{:.2f}".format)
motivo = motivo.astype(float)

for b in pais.columns:
    pais[b] = pd.to_numeric(pais[b], errors="coerce")
pd.set_option("display.float_format", "{:.2f}".format)
pais = pais.astype(float)

In [9]:
#### TEST ####
# turismo.head()
# tipologia.head()
# motivo.head()
# pais.head()
# sector.head()

# Hipotesis

In [10]:
# El turismo en España sigue una fuerte estacionalidad, con picos en verano y descensos en invierno.
# El turismo de ocio y recreo es el más predominante en España frente a otros motivos de viaje como negocios o estudios.
# El turismo de negocios se concentra en ciertas comunidades autónomas más que en otras.
# Los turistas de ciertos países gastan más dinero en España que otros.
# Algunas comunidades autónomas han experimentado un crecimiento más acelerado en el número de turistas que otras.

# Primera hipotesis: ¿El turismo en España sigue una fuerte estacionalidad, con picos en verano y descensos en invierno?

El data set original tiene numeros delante de cada nombre de comunidad autonoma

In [12]:
nombres_turismo = []
for columna in turismo.columns:
    if " " in columna:
        nombres_turismo.append(columna.split(" ", 1)[1])
    else:
        nombres_turismo.append(columna)

turismo.columns = nombres_turismo


En el caso de Madrid, dos columnas tenian la misma informacion pero con diferente nombre.

In [13]:
turismo.drop(["Madrid, Comunidad de"], axis=1, inplace=True) #En el caso de Madrid, dos columnas tenian la misma informacion pero con diferente nombre.

In [14]:
def comunidades(turismo, comunidad):
    # Diccionario de comunidades y provincias
    comunidades_autonomas = {
        "Andalucía": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"],
        "Aragón": ["Huesca", "Teruel", "Zaragoza"],
        "Asturias": ["Asturias"],
        "Balears, Illes": ["Balears, Illes"],
        "Canarias": ["Palmas, Las", "Santa Cruz de Tenerife"],
        "Cantabria": ["Cantabria"],
        "Castilla y León": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"],
        "Castilla - La Mancha": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"],
        "Cataluña": ["Barcelona", "Girona", "Lleida", "Tarragona"],
        "Comunitat Valenciana": ["Alicante/Alacant", "Castellón/Castelló", "Valencia/València"],
        "Extremadura": ["Badajoz", "Cáceres"],
        "Galicia": ["Coruña, A", "Lugo", "Ourense", "Pontevedra"],
        "Madrid": ["Madrid"],
        "Murcia": ["Murcia"],
        "Navarra": ["Navarra"],
        "País Vasco": ["Araba/Álava", "Bizkaia", "Gipuzkoa"],
        "Rioja, La": ["Rioja, La"],
        "Ceuta": ["Ceuta"],
        "Melilla": ["Melilla"]
    }
    provincias = comunidades_autonomas.get(comunidad, [])
    if provincias:
        return turismo.loc[:, turismo.columns.intersection(provincias)]
    else:
        print(f"La comunidad '{comunidad}' no existe.")
        return None

In [15]:
turismo["Año"] = turismo.index.str[:4]  # Toma los primeros 4 caracteres del índice (el año)

# Agrupar por años y sumar los datos
turismo_anual = turismo.groupby("Año").sum()

# Opcional: Redondear los valores
turismo_anual = turismo_anual.round()

# Limpieza del índice del DataFrame Principal

1. Se reemplaza la letra "M" por un espacio para separar el año del mes
2. Se elimina los valores .5, ya que no eran relevantes para representar fechas
3. Se convierte el índice a un formato de fechas utilizando pd.to_datetime con el formato de periodo mensual to_period('M'), dejando solo año y mes.

In [ ]:
turismo.index = turismo.index.astype(str)
turismo.index = turismo.index.str.replace("M", " ") #Reemplaza la M entre año y mes
turismo.index = turismo.index.str.replace(".5", "") #Todos los indices tienen un .5, esto lo elimina
turismo.index = pd.to_datetime(turismo.index).to_period('M') #to_period es para identidicar que solo me coloque el mes y no el 1 por defecto en el dia

/var/folders/_8/pg7m38n54nbc5mpbtzttwghh0000gn/T/ipykernel_9203/1474966129.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  turismo.index = pd.to_datetime(turismo.index).to_period('M') #to_period es para identidicar que solo me coloque el mes y no el 1 por defecto en el dia


In [18]:
turismo.reset_index()

,index,Andalucía,Almería,Cádiz,Córdoba,Granada,Huelva,Jaén,Málaga,Sevilla,...,País Vasco,Araba/Álava,Bizkaia,Gipuzkoa,"Rioja, La","Rioja, La",Ceuta,Melilla,Nacional,Año
0,2025-02,1210103.00,64254.00,127353.00,81664.00,220647.00,37989.00,39187.00,368010.00,270998.00,...,230365.00,28309.00,101349.00,100707.00,34531.00,34531.00,5242.00,4330.00,6539960.00,2025
1,2025-01,983971.00,36982.00,100095.00,76019.00,188585.00,22418.00,33075.00,269769.00,257029.00,...,206997.00,28451.00,86738.00,91808.00,29651.00,29651.00,5044.00,3927.00,5776075.00,2025
2,2024-12,1105005.00,48389.00,123395.00,80679.00,192332.00,35037.00,41265.00,292539.00,291370.00,...,250596.00,36791.00,112602.00,101203.00,34518.00,34518.00,5255.00,4156.00,6553489.00,2024
3,2024-11,1278048.00,49231.00,140183.00,99847.00,208244.00,47220.00,48020.00,369692.00,315609.00,...,278580.00,35718.00,127597.00,115264.00,42997.00,42997.00,5178.00,4864.00,7171221.00,2024
4,2024-10,1838241.00,101499.00,251255.00,110866.00,265193.00,76257.00,58439.00,602997.00,371736.00,...,349844.00,43124.00,157974.00,148746.00,56863.00,56863.00,3920.00,5285.00,10774309.00,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879,1999-05,2000268.00,217067.00,233806.00,63918.00,185433.00,45767.00,13289.00,1033073.00,207915.00,...,70201.00,9621.00,25733.00,34846.00,11952.00,11952.00,2362.00,1305.00,15682588.00,1999
1880,1999-04,1738989.00,122023.00,192676.00,62215.00,187567.00,35056.00,13244.00,921404.00,204805.00,...,64795.00,7804.00,29603.00,27387.00,7917.00,7917.00,2059.00,1668.00,10350825.00,1999
1881,1999-03,1396400.00,116549.00,137098.00,39586.00,149387.00,37433.00,9216.00,758353.00,148778.00,...,44006.00,6095.00,21003.00,16908.00,3907.00,3907.00,3613.00,1673.00,8364216.00,1999
1882,1999-02,942113.00,118444.00,68544.00,18625.00,96391.00,20251.00,4481.00,524335.00,91043.00,...,33581.00,5099.00,16965.00,11517.00,2666.00,2666.00,2312.00,1487.00,6931021.00,1999


In [19]:
turismo["index"]

KeyError: 'index'